# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [ ]:
#Libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score,  precision_score,, recall_score, f1_score


In [36]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [39]:
spaceship.shape

(8693, 14)

In [41]:
spaceship.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [43]:
spaceship.isna().any()

PassengerId     False
HomePlanet       True
CryoSleep        True
Cabin            True
Destination      True
Age              True
VIP              True
RoomService      True
FoodCourt        True
ShoppingMall     True
Spa              True
VRDeck           True
Name             True
Transported     False
dtype: bool

In [45]:
# Removing all the rows with any missing values
cleaned_df = spaceship.dropna()

In [47]:
cleaned_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [49]:
cleaned_df.isna().any()

PassengerId     False
HomePlanet      False
CryoSleep       False
Cabin           False
Destination     False
Age             False
VIP             False
RoomService     False
FoodCourt       False
ShoppingMall    False
Spa             False
VRDeck          False
Name            False
Transported     False
dtype: bool

In [51]:
# Transforming the data of the column 'Cabin' as too granular and obtain {'A', 'B', ...}

def get_first_value(value):
    if isinstance(value, str) and '/' in value: # Checks if the value is string and contains '/'
        new_value = value.split('/')
        if len(new_value) > 0: # Checking if there are at lest 
            return new_value[0]

    return 0

# Applying the above function to update the columns
cleaned_df['Cabin'] = cleaned_df['Cabin'].apply(get_first_value)
cleaned_df['Cabin']

C:\Users\jugal\AppData\Local\Temp\ipykernel_22896\4050190397.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Cabin'] = cleaned_df['Cabin'].apply(get_first_value)


0       B
1       F
2       A
3       A
4       F
       ..
8688    A
8689    G
8690    G
8691    E
8692    E
Name: Cabin, Length: 6606, dtype: object

In [53]:
cleaned_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [55]:
# Dropping passenger ID and Name 
cleaned_df = cleaned_df.drop(['PassengerId', 'Name'], axis = 1)
cleaned_df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [61]:
# Changing the non-numerical columns by using dummies

categorical_col = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']
cleaned_df_transformed = pd.get_dummies(cleaned_df[categorical_col], drop_first=False, dtype=int)
cleaned_df_transformed

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0
1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0
2,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1
3,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0
4,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1
8689,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0
8690,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0
8691,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0


In [63]:
cleaned_df.shape[0]

6606

In [65]:
cleaned_df_transformed.shape[0]

6606

In [67]:
 cleaned_df.shape[0] == cleaned_df_transformed.shape[0]

True

In [69]:
# Joining the data
spaceship_transformed = pd.merge(
    cleaned_df,
    cleaned_df_transformed,
    left_index = True,
    right_index= True

)

spaceship_transformed.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,Europa,False,B,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,1,0
1,Earth,False,F,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,...,0,0,1,0,0,0,0,1,1,0
2,Europa,False,A,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,...,0,0,0,0,0,0,0,1,0,1
3,Europa,False,A,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,...,0,0,0,0,0,0,0,1,1,0
4,Earth,False,F,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,...,0,0,1,0,0,0,0,1,1,0


In [71]:
# Droping the original values of the transfromed columns
spaceship_transformed.drop(columns= ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP'], inplace=True)
spaceship_transformed.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,False,0,1,0,...,0,0,0,0,0,0,0,1,1,0
1,24.0,109.0,9.0,25.0,549.0,44.0,True,1,0,0,...,0,0,1,0,0,0,0,1,1,0
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,0,1,0,...,0,0,0,0,0,0,0,1,0,1
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,0,1,0,...,0,0,0,0,0,0,0,1,1,0
4,16.0,303.0,70.0,151.0,565.0,2.0,True,1,0,0,...,0,0,1,0,0,0,0,1,1,0


In [73]:
# Performing Train test split
X_train, X_test, y_train, y_test = train_test_split(spaceship_transformed.drop(columns=['Transported']), spaceship_transformed['Transported'])

In [75]:
X_train.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
4411,37.0,1144.0,4.0,34.0,0.0,0.0,0,0,1,1,...,1,0,0,0,0,0,0,1,1,0
1241,21.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,...,0,0,0,1,0,1,0,0,1,0
5217,26.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
1941,46.0,430.0,0.0,340.0,52.0,61.0,1,0,0,1,...,0,0,0,1,0,0,0,1,1,0
4486,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,...,0,0,1,0,0,0,0,1,1,0


In [77]:
y_train.head()

4411    False
1241     True
5217     True
1941    False
4486     True
Name: Transported, dtype: bool

In [79]:
X_test.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
2470,23.0,490.0,313.0,0.0,6.0,0.0,1,0,0,1,...,0,0,0,1,0,0,0,1,1,0
5770,33.0,0.0,2110.0,0.0,773.0,10.0,0,1,0,1,...,0,0,0,0,0,0,0,1,1,0
974,61.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,...,0,0,0,1,0,0,0,1,1,0
7593,21.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,...,0,0,0,1,0,0,0,1,1,0
3904,13.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,...,0,0,1,0,0,0,0,1,1,0


In [81]:
y_test.head()

2470    False
5770     True
974      True
7593    False
3904     True
Name: Transported, dtype: bool

In [87]:
# Normalizing
normalizer = MinMaxScaler()
normalizer.fit(X_train)

MinMaxScaler()

In [89]:
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

In [91]:
# When applying transformation of our dataframe, normalizer will retrun an array instead of dataframe
# Therefore, converting to df

pd.DataFrame(X_train_norm, columns=normalizer.feature_names_in_)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,0.468354,0.115323,0.000134,0.002775,0.000000,0.000000,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.265823,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.329114,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.582278,0.043347,0.000000,0.027748,0.003222,0.003525,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4949,0.177215,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4950,0.215190,0.163004,0.000000,0.145026,0.000000,0.000000,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4951,0.227848,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4952,0.291139,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [95]:
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)
X_test_norm.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,0.291139,0.049395,0.010499,0.0,0.000372,0.000000,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.417722,0.000000,0.070774,0.0,0.047896,0.000578,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.772152,0.000000,0.000000,0.0,0.000000,0.000000,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.265823,0.000000,0.000000,0.0,0.000000,0.000000,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.164557,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


Model Selection:

In [112]:
# Create knn
knn = KNeighborsClassifier(n_neighbors=3)

In [114]:
knn.fit(X_train_norm, y_train)

KNeighborsClassifier(n_neighbors=3)

In [148]:
y_pred = knn.predict(X_test)
y_pred[:5]

C:\Users\jugal\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


array([False,  True, False, False,  True])

In [150]:
y_test[:5]

2470    False
5770     True
974      True
7593    False
3904     True
Name: Transported, dtype: bool

In [152]:
knn.score(X_test_norm, y_test)

C:\Users\jugal\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


0.7621065375302664

In [154]:
y_test.shape

(1652,)

In [156]:
y_pred.shape

(1652,)

In [178]:
# 
result = np.logical_xor(y_test, y_pred)


In [ ]:
# mean_squared_error(y_test, y_pred)


In [186]:
# Bagging 
bagging_cls = BaggingClassifier(DecisionTreeClassifier(max_depth=20), n_estimators=100, max_samples=1000)

In [188]:
# Training bagging model with our normalized data
bagging_cls.fit(X_train_norm, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=20),
                  max_samples=1000, n_estimators=100)

In [192]:
# pred = bagging_cls.predict(X_test_norm)
# pred = np.array(pred, dtype = float)
# y_test  = np.array(y_test, dtype=float)

# # Calculate metrics
# mae = mean_absolute_error(y_test, pred)
# mse = mean_squared_error(y_test, pred)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_test, pred)

# # Printing results
# print("MAE:", mae)
# print("RMSE:",rmse)
# print("R2 score:", r2)

MAE: 0.19975786924939468
RMSE: 0.446942803107282
R2 score: 0.20095798272523802


C:\Users\jugal\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but BaggingClassifier was fitted without feature names
  warnings.warn(


In [194]:
bagging_cls.score(X_test_norm, y_test)

C:\Users\jugal\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but BaggingClassifier was fitted without feature names
  warnings.warn(


0.8002421307506054

Random Forests

In [197]:
rf = RandomForestClassifier(n_estimators=100, max_depth=20)

In [199]:
# Training the model
rf.fit(X_train_norm, y_train)

RandomForestClassifier(max_depth=20)

In [205]:
# Evaluating my model
pred = rf.predict(X_test_norm)
pred = np.array(pred, dtype = float)
y_test  = np.array(y_test, dtype=float)

# Calculate metrics
mae = mean_absolute_error(y_test, pred)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, pred)

# Printing results
print("MAE:", mae)
print("RMSE:",rmse)
print("R2 score:", r2)

MAE: 0.19612590799031476
RMSE: 0.4428610481746106
R2 score: 0.21548601940296097


C:\Users\jugal\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [207]:
rf.score(X_test_norm, y_test)

C:\Users\jugal\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


0.8038740920096852

Gradient Boosting

In [210]:
gb_cls = GradientBoostingClassifier(max_depth=20, n_estimators=100)

In [212]:
# Training the model
gb_cls.fit(X_train_norm, y_train)

GradientBoostingClassifier(max_depth=20)

In [214]:
# Evaluating my model
pred = gb_cls.predict(X_test_norm)
pred = np.array(pred, dtype = float)
y_test  = np.array(y_test, dtype=float)

# Calculate metrics
mae = mean_absolute_error(y_test, pred)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, pred)
gb_class_score = gb_cls.score(X_test_norm, y_test)

# Printing results
print("MAE:", mae)
print("RMSE:",rmse)
print("R2 score:", r2)
print("Score:", gb_class_score)

MAE: 0.2148910411622276
RMSE: 0.4635634165486181
R2 score: 0.1404244965680591
Score: 0.7851089588377724


C:\Users\jugal\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(
C:\Users\jugal\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


Adaptive Boosting

In [217]:
ada_cls = AdaBoostClassifier(DecisionTreeClassifier(max_depth=20), n_estimators=100)

In [220]:
# Training the model
ada_cls.fit(X_train_norm, y_train)

C:\Users\jugal\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=20),
                   n_estimators=100)

In [222]:
# Evaluating my model
pred = ada_cls.predict(X_test_norm)
pred = np.array(pred, dtype = float)
y_test  = np.array(y_test, dtype=float)

# Calculate metrics
mae = mean_absolute_error(y_test, pred)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, pred)
ab_class_score = ada_cls.score(X_test_norm, y_test)

# Printing results
print("MAE:", mae)
print("RMSE:",rmse)
print("R2 score:", r2)
print("Score:", ab_class_score)

MAE: 0.22033898305084745
RMSE: 0.46940279403817725
R2 score: 0.11863244155147468
Score: 0.7796610169491526


C:\Users\jugal\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but AdaBoostClassifier was fitted without feature names
  warnings.warn(
C:\Users\jugal\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but AdaBoostClassifier was fitted without feature names
  warnings.warn(


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [ ]:
#your code here

- Evaluate your model

In [1]:
#your code here

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [ ]:
#your code here

- Run Grid Search

- Evaluate your model